<a href="https://colab.research.google.com/github/abdou70/NLP_WEEK1_lab1/blob/main/ABDOU_NIANG_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [2]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [3]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [4]:

print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [32]:
# train_data[1]

In [31]:
# valid_data

In [4]:
# vocab

In [5]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    for sentence in data:
      sentence_unk=[]
      for word in sentence:
        if word in vocab and vocab[word] > mincount:
          sentence_unk.append(word)
        else:
          sentence_unk.append("<unk>")
      data_with_unk.append(sentence_unk)
      #print('*******',data_with_unk)
    return data_with_unk

In [6]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [15]:
# vocab["<unk>"]

5730

In [16]:
# dic={0:{"maman","papa"},1:{'logo','deee'},2:{'fonteh','sari'},3:{'mansour','kkk'},4:{'cameroon','senegal'}}
# print(dic.keys())

dict_keys([0, 1, 2, 3, 4])


In [17]:
# luple=['senegal','mali','Gambie','senegal']
# print(tuple(luple))

('senegal', 'mali', 'Gambie', 'senegal')


In [7]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        '''for word in sentence:
          counts[sentence][word]=sentence.count(word)'''               
        for no in range(len(sentence)):
          total_number_words+=1
          for p in range(n):
            if no-p < 0:
              break
            counts[sentence[no-p:no]][sentence[no]]+=1

    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!
    #print(counts)
    for context in counts.keys():
       div=0
       for word in counts[context].keys():
         div+=counts[context][word]
       for word in counts[context].keys():
         prob[context][word]=counts[context][word]/div
    ## FILL CODE
      
    return prob

In [31]:
# build_ngram(train_data, 3)

In [8]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  2


In [18]:
# model

Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [22]:
# aa={'nom':'abdou','prenom':'abdou','age':14}
# tt=[1,2,3,4,5,6,7]
# print(len(aa))
# for j in range(len(aa)):
#   if j!=2:
#     tt.pop(0)
#   elif j==2:
#     print(tt)

3
[3, 4, 5, 6, 7]


In [9]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    '''prob=1.0
    n=0'''
    ## FILL CODE
    '''if model[context][w]==0:
      for i in range(len(context)):
        if model[context][w]==0:
          context=context[1:]
          n+=1
        elif model[context][w]!=0:
          prob = model[context][w]*n*0.4'''
    if context in model and w in model[context]:
      return model[context][w]
    else:
      return 0.4*get_prob(model,context[1:],w)


In [10]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    '''prob=1.0
    perp=0.0'''
    ## FILL CODE
    '''for sentence in data:
      sentence=tuple(sentence)
      #print('8888',sentence)
      for i in range(len(sentence)-n):
        context=sentence[i:n+i]
        #print('****',context)
        word=context[-1]
        prob*=get_prob(model,context,word)
        #print('####',word)
        #print('????????',model[context][word])
      #   prob*=prob*model[context][word]
      perp=(1/prob)**(len(sentence)-2)'''
    perp,total = 0.0,0
    for sentence in data:
      sentence = tuple(sentence)
      for t in range(1,len(sentence)):
        n_c=min(n-1,t)
        context = sentence[t-n_c:t]
        perp+=-math.log(get_prob(model,context,sentence[t]))
        total+=1
    perp = math.exp(perp/total)
    return perp

In [11]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 53.582886955857006


In [13]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    if context in model:
      return model[context]
    else:
      return get_proba_distrib(model,context[1:])
    # return None

In [16]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        proba = get_proba_distrib(model,tuple(sentence))
        w=np.random.choice((list(proba.keys())) , 1, p = list(proba.values()))
        sentence.append(w[0])
    return sentence

In [17]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', "i'll", 'go', 'to', 'pay', 'more', 'beer', 'with', 'it?', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.